RODDA Ocean Application SANI
Imperial College London - CMCC
Authors: César Quilodrán-Casas and Marco Stefanelli


Loading the data

In [8]:
import xarray as xr
import matplotlib.pyplot as plt
import glob
import pandas as pd
import netCDF4
import scipy
import numpy as np

In [9]:
directory_noDA = '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/'
directory_DA = '/data/c-quilo/CMCC/DA/SHYFEM_ASSIM_BIVARIATE/'
list_of_paths_noDA = sorted(glob.glob(directory_noDA + '/*'))
list_of_paths_DA = sorted(glob.glob(directory_DA + '/*'))


In [16]:
sst_noDA = np.zeros((365*24, 90351))
count = 0
for file in list_of_paths_noDA:
    data_noDA = xr.open_dataset(file)
    sst_noDA[24*count:24*(count+1), :] = data_noDA.temperature
    count += 1
    

In [17]:
sst_DA = np.zeros((365*24, 90351))
count = 0
for file in list_of_paths_DA:
    data_DA = xr.open_dataset(file)
    sst_DA[24*count:24*(count+1)] = data_DA.temperature
    count += 1

In [27]:
print(sst_noDA.shape)

(8760, 90351)


In [28]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [29]:
nFeatures = sst_noDA.shape[1]
encoder_inputs = keras.Input(shape=nFeatures)
alpha = 0.3

enc = keras.layers.Dense(512)(encoder_inputs)
enc = keras.layers.LeakyReLU(alpha)(enc)
enc = keras.layers.Dropout(0.5)(enc)
enc = keras.layers.BatchNormalization()(enc)

enc = keras.layers.Dense(256)(enc)
enc = keras.layers.LeakyReLU(alpha)(enc)
enc = keras.layers.Dropout(0.5)(enc)
enc = keras.layers.BatchNormalization()(enc)

enc = keras.layers.Dense(128)(enc)
enc = keras.layers.LeakyReLU(alpha)(enc)
enc = keras.layers.Dropout(0.5)(enc)
enc = keras.layers.BatchNormalization()(enc)

enc = keras.layers.Dense(256)(enc)
enc = keras.layers.LeakyReLU(alpha)(enc)
enc = keras.layers.Dropout(0.5)(enc)
enc = keras.layers.BatchNormalization()(enc)

enc = keras.layers.Dense(512)(enc)
enc = keras.layers.LeakyReLU(alpha)(enc)
enc = keras.layers.Dropout(0.5)(enc)
enc = keras.layers.BatchNormalization()(enc)

output = keras.layers.Dense(nFeatures)(enc)
#enc = keras.layers.LeakyReLU(alpha)(enc)
#enc = keras.layers.Dropout(0.5)(enc)
#enc = keras.layers.BatchNormalization()(enc)

encoder = keras.Model(encoder_inputs, output, name="encoder")
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 90351)]           0         
_________________________________________________________________
dense_18 (Dense)             (None, 512)               46260224  
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 512)               2048      
_________________________________________________________________
dense_19 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 256)               0   

In [35]:
n_samples = sst_noDA.shape[0]
indices = np.arange(n_samples)
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(sst_noDA, sst_DA, indices, test_size = 0.2, random_state = 42, shuffle=True)
encoder.compile(loss='mse', optimizer=tf.optimizers.Adam())

In [41]:
encoder.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=200, batch_size=256)

Epoch 1/20
28/28 [==============================] - 15s 542ms/step - loss: 9.2758 - val_loss: 19.4843
Epoch 2/20
28/28 [==============================] - 14s 513ms/step - loss: 9.1270 - val_loss: 20.0743
Epoch 3/20
28/28 [==============================] - 15s 531ms/step - loss: 8.6886 - val_loss: 12.9705
Epoch 4/20
28/28 [==============================] - 15s 531ms/step - loss: 8.1774 - val_loss: 6.6388
Epoch 5/20
28/28 [==============================] - 14s 517ms/step - loss: 8.0034 - val_loss: 5.5865
Epoch 6/20
28/28 [==============================] - 15s 534ms/step - loss: 7.5086 - val_loss: 4.3489
Epoch 7/20
28/28 [==============================] - 15s 525ms/step - loss: 7.2364 - val_loss: 3.5661
Epoch 8/20
28/28 [==============================] - 15s 538ms/step - loss: 7.3149 - val_loss: 4.2257
Epoch 9/20
28/28 [==============================] - 15s 537ms/step - loss: 7.2414 - val_loss: 3.3500
Epoch 10/20
28/28 [==============================] - 15s 538ms/step - loss: 7.1055 - val

In [46]:
#Predict DA result
#Read .nc
lon = xr.open_dataset(directory_DA + '/sst_saniv1_chunk_0120.nos.nc').lon
lon =
index = indices_test[42]
plt.tricontourf(lon, lat, sst_DA[index, :], cmap='jet')

NameError: name 'lat' is not defined